# 1. Camera Calibration

**In this script, we intend to obtain an ideal calibration( meaning 0 error) with DLT with normalization and LM optimization.**\
 In order to achieve that it is important to know the calibration matrix in advance. Otherwise, if we generate world points and image points at random with the purpose of finding the projection matrix, the points generate might not have real use correspondence.


When dealing with an uncalibrated camera, 11 parameters are unknown, meaning we need at least **6 points** and assume the model of the camera is **affine**.
>**Problem specification:**
- Estimate the 11 elements of P given:
    - 3D coordinates $X_{i}$ of I $\geq$ 6 object points


<img src="/home/alegria/calibration/images/camera_orientation.png" width=400 height=400 />\


**Import libraries**

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from typing import Sequence
from camera_models import *  # our package

DECIMALS = 2  # how many decimal places to use in print


from utils import *

### Set of Functions

In [2]:
def to_inhomogeneus(X: np.ndarray) -> np.ndarray:
    if X.ndim > 1:
        raise ValueError("x must be one-dimensional.")

    return (X / X[-1])[:-1]


def to_homogeneus(X: np.ndarray) -> np.ndarray:
    if X.ndim > 1:
        raise ValueError("X must be one-dimensional.")

    return np.hstack([X, 1])

def to_homogeneus_arr(X: np.ndarray) -> np.ndarray:
    one_matrix = np.ones((X.shape[0],1))

    out_value = np.concatenate((X, one_matrix), axis=1)

    return out_value

def _get_roll_matrix(theta_x: float = 0.0) -> np.ndarray:
    Rx = np.array(
        [
            [1.0, 0.0, 0.0],
            [0.0, np.cos(theta_x), -np.sin(theta_x)],
            [0.0, np.sin(theta_x), np.cos(theta_x)],
        ]
    )
    return Rx


def _get_pitch_matrix(theta_y: float = 0.0) -> np.ndarray:
    Ry = np.array(
        [
            [np.cos(theta_y), 0.0, np.sin(theta_y)],
            [0.0, 1.0, 0.0],
            [-np.sin(theta_y), 0.0, np.cos(theta_y)],
        ]
    )
    return Ry


def _get_yaw_matrix(theta_z: float = 0.0) -> np.ndarray:
    Rz = np.array(
        [
            [np.cos(theta_z), -np.sin(theta_z), 0.0],
            [np.sin(theta_z), np.cos(theta_z), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )
    return Rz


#my and mx correspond to the pixel density in the x and y direction
def get_calibration_matrix(
    f: float,
    px: float = 0.0,
    py: float = 0.0,
    mx: float = 1.0,
    my: float = 1.0,
) -> np.ndarray:
    K = np.diag([mx, my, 1]) @ np.array([[f, 0.0, px], [0.0, f, py], [0.0, 0.0, 1.0]])
    return K

def get_rotation_matrix(
    theta_x: float = 0.0, theta_y: float = 0.0, theta_z: float = 0.0
) -> np.ndarray:
    # Roll
    Rx = _get_roll_matrix(theta_x)
    # Pitch
    Ry = _get_pitch_matrix(theta_y)
    # Yaw
    Rz = _get_yaw_matrix(theta_z)
    return Rz @ Ry @ Rx

def get_translation_matrix(Tx: float,Ty: float,Tz: float):
    return np.array([[Tx],[Ty],[Tz]])

def get_extrinsic_matrix(rot: np.array((3,3)), trans:np.array((3,1))):
    E = np.concatenate((rot,trans), axis = 1)
    return E

def get_projection_matrix(
    f: float,
    px: float = 0.0,
    py: float = 0.0,
    tx : float = 0.0,
    ty : float = 0.0,
    tz : float = 0.0,
    theta_x: float = 0.0,
    theta_y: float = 0.0,
    theta_z: float = 0.0,
    mx: float = 1.0,
    my: float = 1.0,
) -> np.ndarray:
    K = get_calibration_matrix(f=f, px=px, py=py, mx=mx, my=my)
    #print("Calibration matrix: \n",K)
    R = get_rotation_matrix(theta_x=theta_x, theta_y=theta_y, theta_z=theta_z)
    #print("Rotation matrix: \n",R)
    T = get_translation_matrix(tx,ty,tz)
    #print("Translation matrix: \n",T)
    E = get_extrinsic_matrix(rot = R,trans = T)
    P = K @ E
    
    return P

def get_plucker_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    A = to_homogeneus(A)
    B = to_homogeneus(B)
    L = A.reshape(-1, 1) * B.reshape(1, -1) - B.reshape(-1, 1) * A.reshape(1, -1)
    return L



def normalization(data, dim):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).
    Inputs:
    data: the data to be normalized (directions at different columns and points at rows) -> (nº points, nº coordinates)
    Outputs:
    T : the transformation matrix (translation plus scaling)
    '''

    N = data.shape[0]
    
    if dim == 2:                                                     # 2D points
        x, y = data[:, 0], data[:, 1]                               # x and y coordinates
        x_mean, y_mean = x.mean(), y.mean()                         # mean 
        x_par = (x-x_mean)**2                                       # x part
        y_par = (y-y_mean)**2                                       # y part

        d = (1/N)* np.sum(np.sqrt(x_par + y_par))                   # d
        s_xy = np.sqrt(2)/d                                         # sqrt(2)/d

        T = np.zeros((3,3))                                         # normalization matrix for 2D points
        T[0,0] = s_xy
        T[0,2] = -x_mean*s_xy
        T[1,1] = s_xy
        T[1,2] = -y_mean*s_xy
        T[-1,-1] = 1

        coord_n = np.dot(data,np.transpose(T))                          # normalized image coordinates

    elif(dim== 3):                                                  # 3D points
        X, Y, Z = data[:, 0], data[:, 1], data[:,2]                 # x and y coordinates
        X_mean, Y_mean,Z_mean = X.mean(), Y.mean(),Z.mean()         # mean 
        X_par = (X-X_mean)**2                                       # x part
        Y_par = (Y-Y_mean)**2                                       # y part
        Z_par = (Z-Z_mean)**2                                       # z part

        D = (1/N)* np.sum(np.sqrt(X_par + Y_par + Z_par))           # d
        s_xyz = np.sqrt(3)/D                                        # sqrt(2)/d

        T = np.zeros((4,4))                                         # normalization matrix for 3D points
        T[0,0] = s_xyz
        T[0,3] = -X_mean*s_xyz
        T[1,1] = s_xyz
        T[1,3] = -Y_mean*s_xyz
        T[2,2] = s_xyz
        T[2,3] = -Z_mean*s_xyz
        T[-1,-1] = 1

        coord_n = np.dot(data,T.T)                          # normalized world coordinates

    else:
        raise ValueError('Dataset must be a collection of 2D or points')

    return T,coord_n

def denormalization(H_norm,Txy,Txyz):
    H = np.linalg.pinv(Txy).dot((H_norm).dot(Txyz))
    H = H / H[-1, -2]

    return H


In [3]:
F = 3                                       # focal length( in mm )
sensor_size = np.array([11,7])              # sensor size(mm)
PX= sensor_size[0]/2.0                      # principal point x-coordinate
PY= sensor_size[1]/2.0                      # principal point y-coordinate
IMAGE_HEIGTH = sensor_size[1]
IMAGE_WIDTH = sensor_size[0]
THETA_X = np.pi / 2                         # roll angle
THETA_Z = np.pi                             # yaw angle

C = np.array([6, -5, 2])                    # camera centre


In [4]:
def generate_random_points(n_points, xlim, ylim, zlim):
    '''
    Generate random points in the given limits
    '''
    x = np.random.randint(xlim[0], xlim[1], size=n_points)
    y = np.random.randint(ylim[0], ylim[1], size=n_points)
    z = np.random.randint(zlim[0], zlim[1], size=n_points)
    
    return np.vstack((x, y, z))


In [5]:
def get_image_points(n_points,px,py,trans_x = 0,trans_y = 0,trans_z = 0,F = 3):

    rand_points = generate_random_points(n_points, (-10, 0), (-10, 10), (F, 10))            # generate random world points 
    K = get_calibration_matrix(F, px=px, py=py)                                             # calibration matrix
    P = get_projection_matrix(F, px=px,py =py,tx = trans_x,ty = trans_y, tz = trans_z)                        # projection matrix
    #print("\nCalibration matrix (K):\n", K)
    #print("\nProjection matrix (P):\n", P)

    x = []

    for i in range(n_points):
        Xh = to_homogeneus(rand_points[:,i])
        xh = P @ Xh
        Xx = to_inhomogeneus(xh)
        x.append(Xx)

    x_arr = np.array(x)

    return x_arr, rand_points.T

In [6]:
def SVD(M):
    U, S, Vh = np.linalg.svd(M)

    L = Vh[-1,:] / Vh[-1,-2]
    #Camera projection matrix:
    PEst = L.reshape(3,3+1)
    #print("Camera Matrix:\n",PEst)

    return PEst

In [7]:
def DLT(x_arr,X_arr):

    worldm, worldn = X_arr.shape
    imagem, imagen = x_arr.shape
    
    #3D DLT
    A = []
    world_pts = X_arr
    image_pts = x_arr

    for i in range(worldm):
        A.append([-world_pts[i,0],-world_pts[i,1],-world_pts[i,2],-1,0,0,0,0,image_pts[i,0]*world_pts[i,0],image_pts[i,0]*world_pts[i,1],image_pts[i,0]*world_pts[i,2],image_pts[i,0]])
        A.append([0,0,0,0,-world_pts[i,0],-world_pts[i,1],-world_pts[i,2],-1,image_pts[i,1]*world_pts[i,0],image_pts[i,1]*world_pts[i,1],image_pts[i,1]*world_pts[i,2],image_pts[i,1]])

    # pass the list A as an array
    M = np.asarray(A).reshape(worldm*2,12)
    #print(M.shape)
    
    PEst = SVD(M)
    return PEst

In [42]:
def jac_refine(xdata, *params):
    """Jacobian function for Levenberg-Marquardt refinement.
    """
    h11, h12, h13, h14, h21, h22, h23, h24, h31, h32, h33, h34 = params

    N = xdata.shape[0] // 2

    X = xdata[:,0].T
    Y = xdata[:,1].T
    Z = xdata[:,2].T

    J = np.zeros((N * 2, 12))
    J_x = J[:N]
    J_y = J[N:]

    s_x = h11 * X + h12 * Y + h13*Z + h14
    s_y = h21 * X + h22 * Y + h23*Z + h24
    w   = h31 * X + h32 * Y + h33*Z + h34
    w_sq = w**2

    J_x[:, 0] = X / w
    J_x[:, 1] = Y / w
    J_x[:, 2] = 1. / w
    J_x[:, 6] = (-s_x * X) / w_sq
    J_x[:, 7] = (-s_x * Y) / w_sq
    J_x[:, 8] = -s_x / w_sq

    J_y[:, 3] = X / w
    J_y[:, 4] = Y / w
    J_y[:, 5] = 1. / w
    J_y[:, 6] = (-s_y * X) / w_sq
    J_y[:, 7] = (-s_y * Y) / w_sq
    J_y[:, 8] = -s_y / w_sq

    J[:N] = J_x
    J[N:] = J_y

    return J


In [43]:
def f_refine(xdata, *params):
    """Value function for Levenberg-Marquardt refinement.
    """
    h11, h12, h13, h14, h21, h22, h23, h24, h31, h32, h33, h34 = params


    X = xdata[:,0].T
    Y = xdata[:,1].T
    Z = xdata[:,2].T

    x = (h11 * X + h12 * Y + h13*Z + h14) / (h31 * X + h32 * Y + h33*Z + h34)
    y = (h21 * X + h22 * Y + h23*Z + h24) / (h31 * X + h32 * Y + h33*Z + h34)

    N = X.shape[0]
    print(x.shape)
    print(y.shape)
    result = np.zeros((N*2,))
    result[:N] = x
    result[N:] = y

    return result

In [44]:
def refine_homography(H,worldcoo,imagecoo):
    """
    Performs nonlinear LS to refine linear homography estimates.
    
    Args:
        H : 3x3 homography matrix
        worldcoo : Nx2 world coordinates
        imagecoo : Nx2 image coordinates
    Returns:
        Refined 3x3 homography
    """
    X,Y,Z,x,y = worldcoo[:,0],worldcoo[:,1],worldcoo[:,2],imagecoo[:,0],imagecoo[:,1]
    
    N = X.shape[0]
    
    h0 = H.ravel()

    xdata = np.zeros((N,3))
    xdata[:,0] = X
    xdata[:,1] = Y
    xdata[:,2] = Z
    print(xdata)

    print(xdata.shape)

    ydata = np.zeros((N*2,))
    ydata[:N] = X
    ydata[N:] = Y

    # Use Levenberg-Marquardt to refine the linear homography estimate
    popt, pcov = curve_fit(f_refine, xdata, ydata, p0=h0, jac=jac_refine)
    h_refined = popt
    
    # Normalize and reconstitute homography
    h_refined /= h_refined[-1]
    H_refined = h_refined.reshape((3,3))

    return H_refined

In [45]:
from scipy.optimize import curve_fit

x_arr, X_arr = get_image_points(10,PX,PY)

#print(x_arr.shape)
X_h = to_homogeneus_arr(X_arr)

x_h = to_homogeneus_arr(x_arr)

Txyz, X_h_n = normalization(X_h, 3)
Txy, x_h_n = normalization(x_h, 2)
P_n = DLT(x_h_n,X_h_n)

P_LM = denormalization(P_n,Txy,Txyz)


H_refined = refine_homography(P_LM,X_h,x_h)



[[-10.   7.   5.]
 [ -7.   7.   9.]
 [-10.   2.   5.]
 [ -7.   3.   4.]
 [ -2.  -6.   9.]
 [ -2.   7.   6.]
 [ -7.   0.   7.]
 [ -3.  -8.   5.]
 [ -1.  -1.   9.]
 [ -9.   7.   6.]]
(10, 3)
(10,)
(10,)


ValueError: could not broadcast input array from shape (10,) into shape (5,)

In [11]:
def toHomo(x):
    # converts points from inhomogeneous to homogeneous coordinates
    return np.vstack((x,np.ones((1,x.shape[1]))))

def fromHomo(x):
    # converts points from homogeneous to inhomogeneous coordinates
    return x[:-1,:]/x[-1,:]

def normalize(x):
    dim = x.shape[0]
    # Shift centroid to origin
    m = x.mean(axis=1)
    if dim==3:
        tx = np.array([[1, 0, -m[0]], [0, 1, -m[1]], [0, 0, 1]])
    elif dim==4:
        tx = np.array([[1, 0, 0, -m[0]], [0, 1, 0, -m[1]], [0, 0, 1, -m[2]], [0, 0, 0, 1]])
    
    # Apply translation
    x = tx.dot(x)
    
    # Calculate scaling factor
    var = 0.0
    for i in range(x.shape[1]):
        var+= np.cov(x[:,i])
    s = np.sqrt((x.shape[1]-1)/var)
    #s = np.sqrt((x.shape[1]-1)/np.sum(x.var(axis=1)))
    if dim==3:
        sx = np.array([[s, 0, 0], [0, s, 0], [0, 0, 1]])
    elif dim==4:
        sx = np.array([[s, 0, 0, 0], [0, s, 0, 0], [0, 0, s, 0], [0, 0, 0, 1]])
    
    # Apply scaling
    x = sx.dot(x)
    
    # return normalized coordinate and transformation matrix
    return x, sx.dot(tx)

def householder(x):
    xp = np.zeros((x.shape[1],x.shape[0]-1,x.shape[0]))
    e1 = np.zeros((x.shape[0],1))
    e1[0] = 1.0
    for i in range(x.shape[1]):
        sign = 1 if x[0,i]>=0 else -1
        v = x[:,i].reshape(x.shape[0],1)+sign*np.linalg.norm(x[:,i],2)*e1
        I = np.eye(x.shape[0])
        hv = I - 2*(v.dot(v.T))/(v.T.dot(v))
        xp[i,:,:] = hv[1:,:]
    return xp

def computeP_DLT(x,X):
    # inputs:
    # x 2D points
    # X 3D points
    # output:
    # P_DLT the (3x4) DLT estimate of the camera projection matrix
    
    """your code here"""
    # Convert points to homogeneous space for further processing
    xh = toHomo(x)
    Xh = toHomo(X)
    
    # Normalized the data
    xn, T = normalize(xh)
    Xn, U = normalize(Xh)


    
    # Find orthogonal coordinates rather than taking cross product
    xp = householder(xn)
    print("house:",xp)
    # Construct 2nx12 matrix A    
    A = []
    for i in range(0,X.shape[1]):
        a,b,c,p,q,r = xp[i,:,:].flatten()
        print("",a,b,c,p,q,r)
        xx, yy, zz, ww = Xn[:,i]
        
        # From [xi]⊥PXi=0
        A.append([a*xx, a*yy, a*zz, a*ww, b*xx, b*yy, b*zz, b*ww, c*xx, c*yy, c*zz, c*ww])
        A.append([p*xx, p*yy, p*zz, p*ww, q*xx, q*yy, q*zz, q*ww, r*xx, r*yy, r*zz, r*ww])
        
    A = np.asarray(A).squeeze()
    
    # SVD decomposition for solution
    X, D, Vt = np.linalg.svd(A)
    V = Vt.T
    P = V[:,-1].reshape(3,4)
    
    # Denormalization to convert back to original space
    P_DN = np.linalg.pinv(T).dot((P).dot(U))
    return P_DN/np.sqrt(np.sum(P_DN**2)), P


def proj(P,X):
    # projects 3d points X to 2d using projection matrix P
    return fromHomo(np.matmul(P,toHomo(X)))
    
def rmse(x,y):
    # calculates the root mean square error (RMSE)
    # used to measure reprojection error
    return np.mean(np.sqrt(np.sum((x-y)**2,0)))
    
def displayResults(P, x, X, title):
    print (title+' =')
    print (P)
    print ('||%s||=%f'%(title, np.sqrt(np.sum(P**2)) ))

    x_proj = proj(P,X)
    plt.plot(x[0,:], x[1,:],'.k')
    plt.plot(x_proj[0,:], x_proj[1,:],'.r')
    for i in range(x.shape[1]):
        plt.plot([x[0,i], x_proj[0,i]], [x[1,i], x_proj[1,i]], '-r')
    
    plt.title('mean projection error is %f'%rmse(x,x_proj))
    plt.show()


def sinc(x):
    if x==0:
        return 1.0
    else:
        return np.sin(x)/x
    
def error(x_meas, x_est):
    return x_meas-x_est

def calcCost(error, cov_inv):
    return error.T.dot(cov_inv.dot(error))
    
def jacobian(x, X, P):
    p = parameterize(P)
    #print("P: \n",P)
    #print("p: \n",p)
    pn = np.linalg.norm(p,2)
    if pn==0:
        da = np.zeros((1,p.shape[0]))
        db = 0.5*np.eye(p.shape[0])
    else:
        da = -0.5*deparameterize(p)[1:].T 
        db = 0.5*sinc(pn/2)*np.eye(p.shape[0])+1.0/(4.0*pn)*(np.cos(pn/2)/(pn/2)-np.sin(pn/2)/(pn/2)**2)*p.dot(p.T)
    dpdv = np.concatenate((da,db),axis=0)
    J = np.zeros((2*x.shape[1],p.shape[0]))
    for i in range(x.shape[1]):
        w = P[2,:].dot(X[:,i])
        dxdv = 1.0/w*np.array([[X[0,i], X[1,i], X[2,i], X[3,i], 
                      0, 0, 0, 0, 
                      -x[0,i]*X[0,i], -x[0,i]*X[1,i], -x[0,i]*X[2,i], -x[0,i]*X[3,i]],
                      [X[0,i], X[1,i], X[2,i], X[3,i], 
                      0, 0, 0, 0, 
                      -x[1,i]*X[0,i], -x[1,i]*X[1,i], -x[1,i]*X[2,i], -x[1,i]*X[3,i]]]).reshape(2,-1)
        # chain rule
        J[[2*i,2*i+1],:] = dxdv.dot(dpdv)
    return J

def parameterize(x):
    x = x.flatten()
    x /= np.linalg.norm(x,2)
    a = x[0]
    b = x[1:]
    v = 2.0/sinc(np.arccos(a))*b
    vn = np.linalg.norm(v,2)
    if vn>np.pi:
        v = (1-2*np.pi/vn*np.ceil((vn-np.pi)/(2*np.pi)))*v
    return v.reshape(-1,1)

def deparameterize(v):
    p = np.zeros((v.squeeze().shape[0]+1,1))
    vn = np.linalg.norm(v,2)
    p[0] = np.cos(vn/2)
    p[1:] = sinc(vn/2)/2*v
    return p

def parNorm(v):
    vn = np.linalg.norm(v,2)
    if vn>np.pi:
        v = (1-2*np.pi/vn*np.ceil((vn-np.pi)/(2*np.pi)))*v
    return v.reshape(-1,1)

def LMstep(P, x, X, l, v):
    # inputs:
    # P current estimate of P
    # x 2D points
    # X 3D points
    # l LM lambda parameter
    # v LM change of lambda parameter
    # output:
    # P updated by a single LM step
    # l accepted lambda parameter
    
    """your code here"""
    counter=0
    while(counter<1000):
        counter+=1
        # Calculate current error and cost
        x_estimated = proj(P,fromHomo(X)).T.flatten().reshape(-1,1)
        err = error(x_measured,x_estimated)
        cost = calcCost(err,cov_inv)
        # Calculate jacobian
        J = jacobian(x, X, P)
        subterm1 = J.T.dot(cov_inv.dot(J))
        term1 = np.linalg.inv(subterm1+l*np.eye(subterm1.shape[0]))
        term2 = J.T.dot(cov_inv.dot(err))
        # Parameter update value
        delta = term1.dot(term2)
        p = parameterize(P)
        p_new = parNorm(p + delta)
        # Trying to reproject to check new cost
        P_new = deparameterize(p_new).reshape(3,4)
        x_estimated = proj(P_new,fromHomo(X)).T.flatten().reshape(-1,1)
        err = error(x_measured,x_estimated)
        cost_new = calcCost(err,cov_inv)
        if cost_new<cost:
            return P_new, l/v, err, cost_new
        else:
            l = l*v
    return P, l, err, cost

### Projection matrix estimation

## For multiple n points
Testing the geometric error for n points:
- 10
- 100
- 1000
- 10000

In [12]:
x_arr, X_arr = get_image_points(100,PX,PY)

#print(x_arr.shape)
X_h = to_homogeneus_arr(X_arr)

x_h = to_homogeneus_arr(x_arr)

Txyz, X_h_n = normalization(X_h, 3)
Txy, x_h_n = normalization(x_h, 2)
P_n = DLT(x_h_n,X_h_n)

P_LM = denormalization(P_n,Txy,Txyz)

#print("Camera Matrix:\n",P_LM)



# LM hyperparameters
l=0.001
eta = 10**-6 # for stopping
v=10
max_iters=20
# Data Normalization
xn, T = normalize(toHomo(x_arr.T))
#print(x_arr)
xin = fromHomo(xn)
print(xin.shape)
Xn, U = normalize(toHomo(X_arr.T))
print(Xn.shape)

# Initial cost
x_measured = xin.T.flatten().reshape(-1,1)
x_estimated = proj(P_LM,fromHomo(Xn)).T.flatten().reshape(-1,1)
p = parameterize(P_LM)
scale = T[0,0]**2
cov = scale*np.eye(x_measured.shape[0])
cov_inv = np.linalg.inv(cov)
err = error(x_measured,x_estimated)
cost = calcCost(err,cov_inv)
P_LM = -1*P_LM

costs = []
costs.append(cost)

prev_cost = cost+1

# LM optimization loop
it = 0
while prev_cost-cost>eta:
    prev_cost = cost
    P_LM, l, err, cost = LMstep(P_LM, xin, Xn, l, v)
    P_LMDN = (np.linalg.pinv(T).dot(P_LM)).dot(U)
    P_LMDN = P_LMDN/np.sqrt(np.sum(P_LMDN**2))
    costs.append(cost)
    
    print ('iter %d mean reprojection error %f'%(it+1, rmse(x_arr.T,proj(P_LMDN,X_arr.T))))
    print ('iter %d cost %f'%(it+1, cost))
    it+=1
    
displayResults(P_LMDN, x_arr.T, X_arr.T, 'P_LM')

(2, 100)
(4, 100)
iter 1 mean reprojection error 2.838562
iter 1 cost 1212.213935
iter 2 mean reprojection error 2.826612
iter 2 cost 1206.054864
iter 3 mean reprojection error 2.818969
iter 3 cost 1205.121907


/tmp/ipykernel_24817/3268977799.py:201: RuntimeWarning: invalid value encountered in multiply
  term1 = np.linalg.inv(subterm1+l*np.eye(subterm1.shape[0]))


LinAlgError: SVD did not converge

In [ ]:
n_pts = ([10,100,1000,10000])

err_list = []

for pts in n_pts:   
    x_arr, X_arr = get_image_points(pts,PX,PY)
    X_h = to_homogeneus_arr(X_arr)
    x_h = to_homogeneus_arr(x_arr)

    Txyz, X_h_n = normalization(X_h, 3)
    Txy, x_h_n = normalization(x_h, 2)

    P_n = DLT(x_h_n,X_h_n)

    PEst = denormalization(P_n,Txy,Txyz)

    print("Camera Matrix:\n",PEst)

    n_points = X_h_n.shape[0]
    xEst = np.zeros((n_points,2))

    for j in range(n_points):
        X_pts = to_homogeneus(X_arr[j])
        x_aux  = np.dot(PEst,X_pts)
        xEst[j,:] = to_inhomogeneus(x_aux)
        
    err_list.append(np.sqrt(np.mean(np.sum(np.square(x_arr-xEst), 1))))



print("Geometric error:",err_list)

In [ ]:
err_list